In [ ]:
from nanopyx.liquid import Convolution2D as Conv
from nanopyx.methods import Conv2D_w

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from skimage.transform import resize
import seaborn as sns

input_image = np.random.random((6000,6000)).astype(np.float32)
input_kernel = np.ones((9,9)).astype(np.float32)
max_iter = 500


# Testing liquid implementation

In [ ]:
normal_liquid_time = []
normal_liquid_type = []
for i in range(2):
    out = Conv(clear_benchmarks=True, testing=True).benchmark(input_image,input_kernel)
cw = Conv2D_w(input_image, input_kernel)
for i in range(150):
    output, run_type, run_time = cw.run()
    normal_liquid_time.append(run_time)
    normal_liquid_type.append(run_type)

### Start the delay script here before running next cell

In [ ]:
for i in range(100):
    output, run_type, run_time = cw.run()
    normal_liquid_time.append(run_time)
    normal_liquid_type.append(run_type)

### Stop the delay script

In [ ]:
for i in range(150):
    output, run_type, run_time = cw.run()
    normal_liquid_time.append(run_time)
    normal_liquid_type.append(run_type)

# Testing a non-liquid implementation

In [ ]:
normal_solid_time = []
normal_solid_type = []
cw = Conv2D_w(input_image, input_kernel)
for i in range(150):
    output, run_type, run_time = cw.run(_force_run_type="OpenCL_NVIDIA GeForce RTX 4090")
    normal_solid_time.append(run_time)
    normal_solid_type.append(run_type)

### Start the delay script here before running next cell

In [ ]:
for i in range(100):
    output, run_type, run_time = cw.run(_force_run_type="OpenCL_NVIDIA GeForce RTX 4090")
    normal_solid_time.append(run_time)
    normal_solid_type.append(run_type)

### Stop the delay script

In [ ]:
for i in range(150):
    output, run_type, run_time = cw.run(_force_run_type="OpenCL_NVIDIA GeForce RTX 4090")
    normal_solid_time.append(run_time)
    normal_solid_type.append(run_type)

### Graph the results

In [ ]:
df = pd.DataFrame({'Liquid':normal_liquid_time,'Solid':normal_solid_time})

plt.figure(figsize=(8,3), dpi=300)
ax = sns.histplot(data=df, bins=15, stat='density', kde=True, palette=['magenta','gray'])
plt.axvline(df['Liquid'].mean(), color='magenta', linestyle='dashed', linewidth=1)
plt.axvline(df['Solid'].mean(), color='gray', linestyle='dashed', linewidth=1)
plt.xlabel('Time (s)', fontname='Arial', fontsize=12)
plt.ylabel('Density', fontname='Arial', fontsize=12)
for tick in ax.get_xticklabels():
    tick.set_fontname("Arial")
for tick in ax.get_yticklabels():
    tick.set_fontname("Arial")
plt.setp(ax.get_legend().get_texts(), fontname="Arial", fontsize=12)
ax.spines[['top', 'right']].set_visible(False)
plt.show()

In [ ]:
max_iter=400
methods = {'OpenCL_NVIDIA GeForce RTX 4090':0.5,'Threaded':1.5, 'Threaded_static':1.5, 'Threaded_guided':1.5, 'Threaded_dynamic':1.5,'Unthreaded':2.5}
matrix = np.zeros((max_iter,1))
for i in range(max_iter):
        matrix[i,0] = methods[normal_liquid_type[i]]

matrix_resized = resize(matrix, (max_iter, 30), order=0, preserve_range=True, anti_aliasing=False)

cmap = ListedColormap(["lightskyblue", "coral", "purple"])

fig, ax = plt.subplots(1,1,figsize=(20,20), dpi=300)
p = ax.imshow(matrix_resized.T, cmap=cmap)

ax.set_yticks([])
ax.set_xticks([0,50,100,150,200,250,300,350,400,450,500])

ax.set_xlim((0,max_iter-1))

ax.set_xlabel('# Run', fontname='Arial', fontsize=12)
ax.set_ylabel("Task", fontname='Arial', fontsize=12)


for tick in ax.get_xticklabels():
    tick.set_fontname("Arial")
for tick in ax.get_yticklabels():
    tick.set_fontname("Arial")


In [ ]:
print(df['Liquid'].mean())
print(df['Solid'].mean())